In [33]:
from bs4 import BeautifulSoup
import requests
import os.path
from htmldate import find_date
import pandas as pd
def scrape(url):
    try:
        response = requests.get(url)
    except Exception:
        return "DELETE"
    html_content = response.text
    soup = BeautifulSoup(html_content, 'lxml')  # You can also use 'html.parser'
    paragraphs = soup.find_all("p")
    textOut = ""
    timeStamp = ""
    try:
        timeStamp = find_date(url)
        year = timeStamp.split("-")[0]
        month = timeStamp.split("-")[1]
        day = timeStamp.split("-")[2]
    except Exception:
        year = "check"
        month = "check"
        day = "check"
    
    removeList = []
    authorBio = soup.find_all("div",{"class":"social-author__bio"})
    additionalBio = soup.find("div",{"class":"minibios-group"})
    comments = soup.find_all("div",{"class":"comment--item-text"})
    captions = soup.find_all("div",{"class":"photo-caption"})
    moreCaptions = soup.find_all("small",{"class":"image-media media-caption"})
    for m in moreCaptions:
        removeList.append(m.get_text().replace("\n","").strip())
    miniBios = ""
    for cap in captions:
        removeList.append(cap.get_text().replace("\n","").strip())
    commentsList = []
    for c in comments:
        try:
            commentsList.append(c.get_text().replace("\n","").strip())
        except Exception:
            continue
    if additionalBio != None:
       miniBios = additionalBio.get_text().replace("\n","").strip()
    for a in authorBio:
        removeList.append(a.get_text().replace("\n","").strip())
    isComment = False
    removeList.extend(["This sponsored article is brought to you by BESydney.","Click to read more thoughts from:"])
    for p in paragraphs:
        isComment = False
        text = p.get_text().replace("\n","").strip()
        for comment in commentsList:
            if text in comment:
                isComment = True
        if text != "" and text[-1] in '!.?:”"' and text not in removeList and not isComment and text not in miniBios: #this varies between websites since some write in a more academic manner
            textOut+= " " + text
    length = 0
    if textOut == "":
        return "DELETE"
    else:
        length = len(textOut.split())
        return [url,"Wired",year,month,day,length,textOut]
df = []
